## Create jupyter notebook to replace the user interface (the node "A")

## Try to using widgets to let the user know the position of the robot and all targets that have been set and cancelled in the enviroment.

In [1]:
import rospy
import actionlib
import assignment_2_2023.msg
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2023.msg import msg_struct
from ipywidgets import widgets
from IPython.display import display

# These variables are required to be globally defined
finished = False  # Initialize finished variable globally
client = None  # Initialize client globally
goals = []  # List to store goal positions
cancelled_goals = []  # List to store cancelled goal positions

def publish_msg(msg):
    global publisher
    position = msg.pose.pose.position
    velocity = msg.twist.twist.linear
    new_msg = msg_struct()
    new_msg.x = position.x
    new_msg.y = position.y
    new_msg.vel_x = velocity.x
    new_msg.vel_y = velocity.y
    publisher.publish(new_msg)

def feedback_callback(feedback):
    global finished
    if feedback.stat == "Target reached!":
        finished = True
        print("Target reached!")
        update_goal_info()

def cancel_goal(button):
    global finished, client, cancelled_goals
    if client and not finished:  # Check if client is not None
        if goals:  # Check if there are goals to cancel
            cancelled_goal = goals.pop()
            cancelled_goals.append(cancelled_goal)
            print("Goal has been cancelled!")
        else:
            print("No goals to cancel!")

        client.cancel_goal()
        finished = True
        update_cancelled_goal_info()

def start_client(button, x_input, y_input):
    global finished, client, goals
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
    client.wait_for_server()

    x = x_input.value
    y = y_input.value
    
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y

    finished = False
    client.send_goal(goal, None, None, feedback_callback)
    
    # Add the goal position to the list
    goals.append((x, y))
    update_goal_info()

def on_execute_button_clicked(button, x_input, y_input):
    start_client(button, x_input, y_input)

def update_goal_info():
    global goals, goal_info
    goal_info.value = ""
    for i, goal in enumerate(goals, start=1):
        goal_info.value += f"Goal {i}: ({goal[0]}, {goal[1]})\n"

def update_cancelled_goal_info():
    global cancelled_goals, cancelled_goal_info
    cancelled_goal_info.value = ""
    for i, goal in enumerate(cancelled_goals, start=1):
        cancelled_goal_info.value += f"Goal {i}: ({goal[0]}, {goal[1]})\n"

def main():
    global publisher, goal_info, cancelled_goal_info
    rospy.init_node('action_client')
    publisher = rospy.Publisher("/position_velocity", msg_struct, queue_size=1)
    rospy.Subscriber("/odom", Odometry, publish_msg)
    
    instruction_text1 = widgets.HTML(value="<h4>Please, provide the coordinates:</h4>")

    x_input = widgets.BoundedIntText(value=0, min=0, max=100, bar_style='info')
    y_input = widgets.BoundedIntText(value=0, min=0, max=100, bar_style='info')
    
    execute_button = widgets.Button(value=False, 
                                    description="Set Goal", 
                                    disabled=False, 
                                    button_style='info',
                                    icon='play')
    execute_button.on_click(lambda b: on_execute_button_clicked(b, x_input, y_input))
    
    instruction_text2 = widgets.HTML(value="<h4>Press the button if you want to cancel the goal: </h4>")

    cancel_button = widgets.Button(value=False, 
                                   description="Cancel Goal",
                                   disabled=False,
                                   button_style='danger',
                                   icon='times')
    cancel_button.on_click(cancel_goal)
    
    goal_info = widgets.Textarea(
        value="",
        description="Setted:",
        disabled=True,
        
    )
    
    cancelled_goal_info = widgets.Textarea(
        value="",
        description="Cancelled:",
        disabled=True
    )

    display(widgets.VBox([instruction_text1, widgets.HBox([x_input, y_input]), execute_button, 
                          instruction_text2, cancel_button, goal_info, cancelled_goal_info]))

if __name__ == "__main__":
    main()


Goal has been cancelled!
